In [1]:
import pandas as pd
import metadata_parser as mp
import requests
from bs4 import BeautifulSoup as bs
import re
import undetected_chromedriver as uc
import time

In [2]:
# Call dataset
ieee = pd.read_excel("./paper-links/PaperLinks_IEEEXplore.xlsx")

In [ ]:
# Get the paper metadata
list_year, list_title, list_conf_jour, list_authors, list_keywords, list_abstract = [], [], [], [], [], []
driver = uc.Chrome()
index = 1
for paper_url in ieee["Paper Links"]:
    try:
        # Get page
        page = mp.MetadataParser(paper_url)
        page_bs = requests.get(paper_url)
        time.sleep(5)
        parsed_bs = bs(page_bs.text,'html.parser')
        # Get Year
        try:
            ls_year = str(parsed_bs).split('publicationDate')
            year_dirty = ls_year[2].split('","')
            year = year_dirty[0].split(" ")[1]
        except:
            year = "failed_to_scrap"   
        # Get Title
        try:
            title = page.get_metadatas('og:title')
            title = title[0]
        except:
            title = "failed_to_scrap" 
        # Get Conf/Journal Name
        try:
            ls_conf_jour = str(parsed_bs).split('publicationTitle')
            conf_jour_dirty = ls_conf_jour[1].split('","')
            conf_jour_name = re.sub("[\:\"]", "", conf_jour_dirty[0]).strip()
        except:
            conf_jour_name = "failed_to_scrap"
        # Get Authors
        authors_url = paper_url+"authors#authors"
        try:
            driver.get(authors_url)
            time.sleep(10)
            try:
                driver.find_element_by_css_selector("[aria-label='dismiss cookie message'").click()
                time.sleep(5)
            except:
                pass
            get_element = driver.find_elements_by_css_selector("div[class='authors-accordion-container']")
            authors = "failed_to_scrap" # Just in case failed to find the class
            for i in range(0,len(get_element)):
                if i == 0:
                    authors = get_element[i].text.split("\n")[0]
                else:
                    authors = authors+", "+get_element[i].text.split("\n")[0]
        except:
            authors = "failed_to_scrap" 
        # Get Keywords
        try:
            ls_key = str(parsed_bs).split('{"type":"Author Keywords ","kwd":')
            key_dirty = ls_key[1].split("]}]")
            keywords = re.sub("[\[\"]", "", key_dirty[0]).strip()
            keywords = re.sub(",", ", ", keywords).strip()
        except:
            keywords = "failed_to_scrap"    
        # Get Abstract
        try:
            abstract = page.get_metadatas('og:description')
            abstract = abstract[0]
        except:
            abstract = "failed_to_scrap"
    except:
        year = "failed_to_scrap"
        title = "failed_to_scrap"
        conf_jour_name = "failed_to_scrap"
        authors = "failed_to_scrap"
        keywords = "failed_to_scrap"
        abstract = "failed_to_scrap"
        
    # Append to the list
    list_year.append(year)
    list_title.append(title)
    list_conf_jour.append(conf_jour_name)
    list_authors.append(authors)
    list_keywords.append(keywords)
    list_abstract.append(abstract)
    print(index)
    index += 1

driver.quit()

In [8]:
# Concatenate list as new column
ieee["Year"], ieee["Title"], ieee["Conf/Journal Name"] = list_year, list_title, list_conf_jour
ieee["Authors"], ieee["Keywords"], ieee["Abstract"] = list_authors, list_keywords, list_abstract

In [9]:
ieee.to_excel("./paper-metadata/PaperMetadata_Unclean_IEEEXplore.xlsx", index = False)